In [74]:
# Import Libraries

import pandas as pd
pd.set_option('display.max_columns', 500)
import arcgis
from arcgis.gis import GIS
from arcgis.geocoding import batch_geocode, geocode
import dask.dataframe as dd

In [75]:
# Create GIS connection

gis = GIS("https://www.arcgis.com","arcgis_python","P@ssword123")

In [88]:
# Read the data as dask dataframe

# ca_data = dd.read_csv('CA_Sample2.csv.csv', blocksize=100000, dtype=str)
# ca_data = dd.read_csv('CA_Sample2.csv', dtype=str)
npi_data = dd.read_csv('npidata.csv', dtype=str)

In [89]:
# Confirm type of dataset
type(npi_data)

dask.dataframe.core.DataFrame

In [90]:
npi_data.head()

,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Business Mailing Address Telephone Number,Provider Business Mailing Address Fax Number,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Country Code (If outside U.S.),Provider Business Practice Location Address Telephone Number,Provider Business Practice Location Address Fax Number,Provider Enumeration Date,Last Update Date,NPI Deactivation Reason Code,NPI Deactivation Date,NPI Reactivation Date,Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Healthcare Provider Taxonomy Code_1,Provider License Number_1,Provider License Number State Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Provider License Number_2,Provider License Number State Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Provider License Number_3,Provider License Number State Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Provider License Number_4,Provider License Number State Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Provider License Number_5,Provider License Number State Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Provider License Number_6,Provider License Number State Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Provider License Number_7,Provider License Number State Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Provider License Number_8,Provider License Number State Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Provider License Number_9,Provider License Number State Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Provider License Number_10,Provider License Number State Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Provider License Number_11,Provider License Number State Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Provider License Number_12,Provider License Number State Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Provider License Number_13,Provider License Number State Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Provider License Number_14,Provider License Number State Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Provider License Number_15,Provider License Number State Code_15,Healthcare Provider Primary Taxonomy Switch_15,Other Provider Identifier_1,Other Provider Identifier Type Code_1,Other Provider Ide

### Clean Columns

In [91]:
# Identify columns to keep
cols_to_keep = ['NPI','Entity Type Code1', 'Healthcare Provider Taxonomy Code 1', 'Healthcare Provider Taxonomy Code 2',\
                   'Healthcare Provider Taxonomy Code 3','Healthcare Provider Taxonomy Code 4', 'Healthcare Provider Taxonomy Code 5',\
                   'Healthcare Provider Taxonomy Code 6','Healthcare Provider Taxonomy Code 7','Healthcare Provider Taxonomy Code 8',\
                   'Healthcare Provider Taxonomy Code 9','Healthcare Provider Taxonomy Code 10','Healthcare Provider Taxonomy Code 11',\
                   'Healthcare Provider Taxonomy Code 12','Healthcare Provider Taxonomy Code 13','Healthcare Provider Taxonomy Code 14',\
                    'Healthcare Provider Taxonomy Code 15','Healthcare Provider Taxonomy Group 1',\
                    'Provider Gender Code','Provider Organization Name (Legal Business Name)',\
                    'Provider First Line Business Practice Location Address1',\
                   'Provider Second Line Business Practice Location Address','Provider Business Practice Location Address City Name1',\
                    'Provider Business Practice Location Address State Name','Provider Business Practice Location Address Country Code (If outside U.S.)',\
                   'Provider Business Practice Location Address Postal Code1']

In [92]:
# Get columns to drop
all_columns = set(npi_data.columns)
cols_to_keep_set = set(cols_to_keep)
cols_to_drop = all_columns - cols_to_keep_set

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [81]:
# cols_to_drop = len(all_columns) - len(cols_to_keep_set)
# cols_to_drop

In [82]:
len(all_columns), len(cols_to_keep_set), len(cols_to_drop)

(329, 26, 323)

In [72]:
# Drop columns
npi_data = npi_data.drop(list(cols_to_drop), axis=1)

TypeError: 'int' object is not iterable

In [73]:
npi_data.head()

,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Business Mailing Address Telephone Number,Provider Business Mailing Address Fax Number,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Country Code (If outside U.S.),Provider Business Practice Location Address Telephone Number,Provider Business Practice Location Address Fax Number,Provider Enumeration Date,Last Update Date,NPI Deactivation Reason Code,NPI Deactivation Date,NPI Reactivation Date,Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Healthcare Provider Taxonomy Code_1,Provider License Number_1,Provider License Number State Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Provider License Number_2,Provider License Number State Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Provider License Number_3,Provider License Number State Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Provider License Number_4,Provider License Number State Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Provider License Number_5,Provider License Number State Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Provider License Number_6,Provider License Number State Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Provider License Number_7,Provider License Number State Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Provider License Number_8,Provider License Number State Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Provider License Number_9,Provider License Number State Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Provider License Number_10,Provider License Number State Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Provider License Number_11,Provider License Number State Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Provider License Number_12,Provider License Number State Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Provider License Number_13,Provider License Number State Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Provider License Number_14,Provider License Number State Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Provider License Number_15,Provider License Number State Code_15,Healthcare Provider Primary Taxonomy Switch_15,Other Provider Identifier_1,Other Provider Identifier Type Code_1,Other Provider Ide

In [112]:
# Rename Columns
ca_data = ca_data.rename(columns={"Entity Type Code1":"entity_type",'Provider First Line Business Practice Location Address1':'addressline1',
                'Healthcare Provider Taxonomy Code 1':'taxonomy_code_1','Healthcare Provider Taxonomy Code 2':'taxonomy_code_2',
                'Healthcare Provider Taxonomy Code 3':'taxonomy_code_3','Healthcare Provider Taxonomy Code 4':'taxonomy_code_4',
                'Healthcare Provider Taxonomy Code 5':'taxonomy_code_5','Healthcare Provider Taxonomy Code 6':'taxonomy_code_6',
                'Healthcare Provider Taxonomy Code 7':'taxonomy_code_7','Healthcare Provider Taxonomy Code 8':'taxonomy_code_8',
                'Healthcare Provider Taxonomy Code 9':'taxonomy_code_9','Healthcare Provider Taxonomy Code 10':'taxonomy_code_10',
                'Healthcare Provider Taxonomy Code 11':'taxonomy_code_11','Healthcare Provider Taxonomy Code 12':'taxonomy_code_12',
                'Healthcare Provider Taxonomy Code 13':'taxonomy_code_13','Healthcare Provider Taxonomy Code 14':'taxonomy_code_14',
                'Healthcare Provider Taxonomy Code 15':'taxonomy_code_15','Healthcare Provider Taxonomy Group 1':'taxonomy_group_1',
                'Provider Gender Code':'provider_gender','Provider Organization Name (Legal Business Name)':'organization_name',
                'Provider Second Line Business Practice Location Address':'addressline2',
                'Provider Business Practice Location Address City Name1':'city',
                'Provider Business Practice Location Address State Name':'state',
                'Provider Business Practice Location Address Country Code (If outside U.S.)':'country',
                'Provider Business Practice Location Address Postal Code1':'postal_code','NPI':'npi'})

In [114]:
ca_data.head()

,npi,organization_name,addressline2,state,country,provider_gender
0,1679576722,NaN,NaN,NE,US,M
1,1588667638,NaN,SUITE 300,FL,US,M
2,1497758544,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NC,US,NaN
3,1306849450,NaN,NaN,TX,US,M
4,1215930367,NaN,NaN,TX,US,M


In [99]:
# ca_data[ca_data['country']=='US']['state'].value_counts().compute()
valuedf = ca_data[ca_data['country']=='US']['state'].value_counts().compute()

In [102]:
valuedf[:50]

CA    678482
NY    467416
FL    375603
TX    365783
PA    233208
OH    227028
IL    215406
MI    202239
MA    175534
NC    174073
WA    154939
NJ    153089
GA    140457
VA    121950
MD    118635
CO    114684
AZ    108769
TN    105101
MO     99871
MN     99159
WI     99088
OR     97492
IN     90921
LA     84271
OK     74894
KY     74888
SC     74353
CT     71388
AL     61092
NV     59619
AR     57182
UT     51494
KS     48582
NM     45404
IA     44272
PR     41198
MS     39325
NE     34754
DC     32168
ME     30334
WV     29431
NH     27367
ID     27335
HI     24848
RI     21583
AK     21012
MT     19801
DE     18269
SD     15635
ND     15382
Name: state, dtype: int64

In [103]:
valuedf[50:]

VT               12840
WY               12770
AE                1875
AP                1440
VI                 875
GU                 813
AA                 245
MP                 224
AS                 124
PW                  40
FM                  10
US                   5
ITALY                2
MH                   2
UNITED STATES        2
FP                   1
FPO                  1
GERMANY              1
GUAM                 1
KOREA                1
US NAVY              1
USA                  1
MILITARY             1
Name: state, dtype: int64

In [113]:
# Create new address column
ca_data['full_address'] = ca_data['addressline1'].astype(str)+', '+ca_data['addressline2'].fillna('').astype(str)+', '\
    +ca_data['city'].astype(str)+', '+ca_data['state'].astype(str)+' '+ca_data['postal_code'].astype(str)

KeyError: 'addressline1'

#### Change categories of Entity Types

In [42]:
ca_data['entity_type'] = ca_data['entity_type'].mask(ca_data['entity_type'] == '1', 'Individual')
                        .mask(ca_data['entity_type'] == '2', 'Organization')

In [43]:
ca_data['entity_type'] = ca_data['entity_type'].mask(ca_data['entity_type'] == '2', 'Organization')

In [44]:
# Check the dataset
ca_data[ca_data['entity_type']=='Organization'].head()

,entity_type,taxonomy_code_10,taxonomy_code_11,taxonomy_code_12,taxonomy_code_13,taxonomy_code_14,taxonomy_code_15,taxonomy_code_1,taxonomy_code_2,taxonomy_code_3,taxonomy_code_4,taxonomy_code_5,taxonomy_code_6,taxonomy_code_7,taxonomy_code_8,taxonomy_code_9,taxonomy_group_1,npi,city,country,postal_code,state,addressline1,provider_gender,organization_name,addressline2,full_address
21,Organization,NaN,NaN,NaN,NaN,NaN,NaN,251G00000X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023011178,NAPA,US,945594515,CA,414 S JEFFERSON ST,NaN,COLLABRIA CARE,NaN,"414 S JEFFERSON ST, , NAPA, CA 945594515"
22,Organization,NaN,NaN,NaN,NaN,NaN,NaN,3336C0003X,3336S0011X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023011053,LOS ANGELES,US,900381662,CA,1253 N VINE ST,NaN,VINE DISCOUNT PHARMACY & MEDICAL SUPPLY,STE 11,"1253 N VINE ST, STE 11, LOS ANGELES, CA 900381662"
27,Organization,NaN,NaN,NaN,NaN,NaN,NaN,282N00000X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1134122179,CASTRO VALLEY,US,945465305,CA,20103 LAKE CHABOT RD,NaN,EDEN MEDICAL CENTER,NaN,"20103 LAKE CHABOT RD, , CASTRO VALLEY, CA 9454..."
29,Organization,NaN,NaN,NaN,NaN,NaN,NaN,251E00000X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1679576607,SIMI VALLEY,US,930656508,CA,1687 ERRINGER RD,NaN,"GENESIS HOME HEALTH, INC",STE 202,"1687 ERRINGER RD, STE 202, SIMI VALLEY, CA 930..."
31,Organization,NaN,NaN,NaN,NaN,NaN,NaN,282J00000X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1417950445,LOS ANGELES,US,900311422,CA,4570 GRIFFIN AVE,NaN,"BROADVIEW, INC.",NaN,"4570 GRIFFIN AVE, , LOS ANGELES, CA 900311422"


In [36]:
ca_data.visualize

<bound method DaskMethodsMixin.visualize of Dask DataFrame Structure:
              entity_type taxonomy_code_10 taxonomy_code_11 taxonomy_code_12 taxonomy_code_13 taxonomy_code_14 taxonomy_code_15 taxonomy_code_1 taxonomy_code_2 taxonomy_code_3 taxonomy_code_4 taxonomy_code_5 taxonomy_code_6 taxonomy_code_7 taxonomy_code_8 taxonomy_code_9 taxonomy_group_1     NPI    city country postal_code   state addressline1 provider_gender organization_name addressline2 full_address
npartitions=1                                                                                                                                                                                                                                                                                                                                                                                                        
                   object           object           object           object           object           object        

In [15]:
import graphviz
graphviz.__file__

'C:\\Users\\mohi9282\\AppData\\Local\\Continuum\\anaconda3\\envs\\arcgis\\lib\\site-packages\\graphviz\\__init__.py'

In [18]:
graphviz.__path__

['C:\\Users\\mohi9282\\AppData\\Local\\Continuum\\anaconda3\\envs\\arcgis\\lib\\site-packages\\graphviz']

### Geocode

In [50]:
# Define function to geocode addresses
# def geocode_records(row):
#     geoAddress = geocode(row['Full_Address'])
#     row['Longitude'] = geoAddress[0]['location']['x']
#     row['Latitude'] = geoAddress[0]['location']['y']
#     return row

In [51]:
# # Apply geocode function
# ca_data = ca_data.apply(geocode_records, axis=1)

# # Check the dataset
# #ca_data.head()

C:\Users\mohi9282\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\dataframe\core.py:3121: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


### Compute the dataframe

In [37]:
new_df = ca_data.compute()

In [38]:
new_df.shape

(249, 27)

In [39]:
new_df.describe()

,entity_type,taxonomy_code_10,taxonomy_code_11,taxonomy_code_12,taxonomy_code_13,taxonomy_code_14,taxonomy_code_15,taxonomy_code_1,taxonomy_code_2,taxonomy_code_3,taxonomy_code_4,taxonomy_code_5,taxonomy_code_6,taxonomy_code_7,taxonomy_code_8,taxonomy_code_9,taxonomy_group_1,NPI,city,country,postal_code,state,addressline1,provider_gender,organization_name,addressline2,full_address
count,50,0.0,0.0,0.0,0.0,0.0,0.0,50,8,4,2,1,1,1,0.0,0.0,1,50,50,50,50,50,50,44,6,25,249
unique,2,0.0,0.0,0.0,0.0,0.0,0.0,34,8,4,2,1,1,1,0.0,0.0,1,50,37,1,48,1,48,2,6,23,49
top,Individual,NaN,NaN,NaN,NaN,NaN,NaN,208000000X,207N00000X,207N00000X,207N00000X,207N00000X,207N00000X,207N00000X,NaN,NaN,193200000X MULTI-SPECIALTY GROUP,1821091836,SACRAMENTO,US,958211619,CA,600 COFFEE RD,F,"GENESIS HOME HEALTH, INC",SUITE 204,"nan, , nan, nan nan"
freq,44,NaN,NaN,NaN,NaN,NaN,NaN,4,1,1,1,1,1,1,NaN,NaN,1,1,4,50,2,50,2,33,1,2,199


In [39]:
export_csv = new_df.to_csv (r'C:\Users\mohi9282\Desktop\ca_export.csv', index = None, header=True)

### Plot on Map

In [333]:
sdf = pd.DataFrame.spatial.from_xy(new_df, x_column='Longitude', y_column='Latitude')

In [334]:
map1 = gis.map()
map1

MapView(layout=Layout(height='400px', width='100%'))

In [335]:
map1.draw(sdf.spatial.to_featureset())

In [119]:
!pip install graphviz

In [120]:
ca_data.visualize()

ExecutableNotFound: failed to execute ['dot', '-Tpng'], make sure the Graphviz executables are on your systems' PATH